# Set-up


In [ ]:
# mount to google drive
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.chdir('./drive/MyDrive/Project')

In [ ]:
!pip install simplet5
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install rouge_score
!pip install sacrebleu
!pip install accelerate -U

In [ ]:
import sacrebleu
import rouge_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch

import nltk
import evaluate

from datasets import load_dataset, DatasetDict, Dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
import tqdm
from tqdm.auto import tqdm
from simplet5 import SimpleT5

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


In [ ]:
torch.cuda.empty_cache()

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
# load data
df_train = pd.read_csv('./train_datasets/combined_train.csv')
df_test = pd.read_csv('./test_datasets/combined_test.csv')

# Model

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=123)

train_df = train_df.rename(columns={'Question': 'source_text', 'Answer_cut': 'target_text'})
val_df = val_df.rename(columns={'Question': 'source_text', 'Answer_cut': 'target_text'})

train_df['source_text'] = "[Question] " + train_df['source_text']
val_df['source_text'] = "[Question] " + val_df['source_text']

In [ ]:
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-small")

Downloading:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [ ]:
model.train(train_df = train_df,
            eval_df = val_df,
            source_max_token_len=200,
            target_max_token_len=200,
            batch_size=32,
            max_epochs=3,
            use_gpu=True)

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# Evaluation

In [ ]:
model = SimpleT5()
model.load_model("t5", "./outputs/combined-t5-checkpoints", use_gpu=torch.cuda.is_available())

tokenizer = T5Tokenizer.from_pretrained("./outputs/combined-t5-checkpoints")
model = T5ForConditionalGeneration.from_pretrained("./outputs/combined-t5-checkpoints")
model.to(device)

df_test_combined = df_test

def generate_answers_batch(questions, batch_size=32):
    # initialize the answer list generated by batch processing
    batch_generated_answers = []
    for i in tqdm(range(0, len(questions), batch_size), desc="Generating answers"):
        batch_questions = ["[Question] " + q for q in questions[i:i+batch_size]]
        batch_inputs = tokenizer(batch_questions, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
        with torch.no_grad():
            batch_outputs = model.generate(**batch_inputs, max_length=500,
                                           min_length=50,
                                           length_penalty=2.0)
        batch_answers = [tokenizer.decode(output, skip_special_tokens=True) for output in batch_outputs]
        batch_generated_answers.extend(batch_answers)
    return batch_generated_answers



generated_answers = generate_answers_batch(df_test_combined['Question'].tolist())

df_test_combined['Generated_Answer'] = generated_answers

df_test_combined.to_csv("./outputs/combinedGenerated_Answer.csv", index = False)



Generating answers:   0%|          | 0/289 [00:00<?, ?it/s]

In [ ]:
model = SimpleT5()
model.load_model("t5", "./outputs/combined-t5-checkpoints", use_gpu=torch.cuda.is_available())

q_test = df_test['Question'][115]
q_ans = df_test['Answer'][115]

print("Question: ", q_test)
print('-'*50)
print("Reference Answer: ",q_ans)
predicted_ans = model.predict(q_test)[0]
print("Predicted Answer: " ,predicted_ans)

Question:  What is (are) Late-Onset Familial Alzheimer Disease ?
--------------------------------------------------
Reference Answer:  Lateonset familial Alzheimer disease is a form of familial Alzheimer disease that begins after age 65. In general, Alzheimer disease (AD) is a degenerative disease of the brain that causes gradual loss of memory, judgement and the ability to function socially. The exact underlying cause of lateonset familial AD is not completely understood; however, researchers suspect that it is a complex condition, which is likely associated with multiple susceptibility genes (such as the APOE e4 allele) in combination with environmental and lifestyle factors. Although complex conditions do tend to cluster in families, they do not follow a clearcut pattern of inheritance. There is no cure for AD. Treatment is supportive and based on the signs and symptoms present in each person.
Predicted Answer:  LateOnset Familial Alzheimer disease is a condition that affects the br

In [ ]:
references = [[ref_ans] for ref_ans in df_test_combined['Answer']]
predictions = [pre_ans for pre_ans in df_test_combined['Generated_Answer']]


sacrebleu = evaluate.load("sacrebleu")
sacrebleu_results = sacrebleu.compute(predictions=predictions, references=references)
print(f"SacreBLEU Results: {sacrebleu_results}")

rouge = evaluate.load("rouge")
rouge_results = rouge.compute(predictions=predictions, references=references)
print(f"ROUGE Results: {rouge_results}")


SacreBLEU Results: {'score': 4.568909720904612, 'counts': [235315, 84971, 59238, 51167], 'totals': [1945017, 1935788, 1926559, 1917330], 'precisions': [12.098351839598317, 4.3894785999293315, 3.074808505734836, 2.6686590206172123], 'bp': 1.0, 'sys_len': 1945017, 'ref_len': 1328123}


ROUGE Results: {'rouge1': 0.1391344909419368, 'rouge2': 0.05667095864018188, 'rougeL': 0.12019552350954973, 'rougeLsum': 0.12025811282776791}
